# 유튜브 급상승 동영상 분석을 통한 미국과 일본에서의 한국 컨텐츠 관심도


**`1-1 왜 유튜브일까요?`**

유튜브는 전 세계적으로 가장 인기 있는 온라인 비디오 플랫폼 중 하나로, 그 사용자 수는 지속적으로 증가하고 있습니다. 2023년 현재, 전 세계적으로 매월 약 **27억 명이 유튜브를 사용**하는 것으로 **추정**됩니다. 이러한 증가 추세는 디지털 미디어의 사용이 증가하고, 비디오 콘텐츠에 대한 수요가 계속해서 높아지고 있음을 나타냅니다. 또한 유튜브는 단순한 비디오 공유 플랫폼을 넘어, **중요한 검색 엔진으로서의 역할**을 하고 있습니다. 사용자들은 유튜브를 통해 다양한 주제에 대한 정보를 검색하고, 교육, 오락, 뉴스 등 다양한 형태의 콘텐츠를 소비합니다. 이는 유튜브가 단순한 엔터테인먼트 채널을 넘어서, **정보 검색 및 학습의 중심지로 자리 잡고 있음을 보여줍니다**.


https://www.globalmediainsight.com/blog/youtube-users-statistics/#:~:text=Interesting%20YouTube%20Statistics%202023,largest%20search%20engine%2C%20after%20Google


유튜브 검색 활동은 **전 세계적인 트렌드와 관심사를 파악하는 데 중요한 지표**가 됩니다. 특히, 유튜브의 급상승 동영상은 현재 가장 인기 있는 주제와 콘텐츠 유형을 반영하며, 이를 통해 사용자들의 관심과 행동 패턴을 분석할 수 있습니다.

**1-2 데이터는 어디에서 구해올까요?**

https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset 의 데이터를 가지고 활용하였습니다.

**2-1 US 인기 급상승 동영상 데이터 정제하기**

In [ ]:

import os
os.kill(os.getpid(), 9)

In [ ]:
pip install unidecode

In [ ]:
pip install chardet

In [ ]:
import pandas as pd

# 판다스 버전 확인
print("Pandas 버전:", pd.__version__)


In [38]:
import pandas as pd

df = pd.read_csv("/content/US_youtube_trending_data.csv",sep=',',
                 engine='python',
                 header=0,encoding="UTF-8",error_bad_lines=False)
# 데이터프레임 확인
print(df.head())
df.columns

<ipython-input-38-0ec507336528>:3: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.



Skipping line 142806: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 143034: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 143276: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 143508: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 143743: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 143973: NULL byte detected. This by

      video_id                                              title  \
0  3C66w5Z0ixs                 I ASKED HER TO BE MY GIRLFRIEND...   
1  M9Pmf9AB4Mo  Apex Legends | Stories from the Outlands – “Th...   
2  J78aPJ3VyNs  I left youtube for a month and THIS is what ha...   
3  kXLn3HkpjaA  XXL 2020 Freshman Class Revealed - Official An...   
4  VIUo6yapDbc  Ultimate DIY Home Movie Theater for The LaBran...   

            publishedAt                 channelId   channelTitle  categoryId  \
0  2020-08-11T19:20:14Z  UCvtRTOMP2TqYqu51xNrqAzg       Brawadis          22   
1  2020-08-11T17:00:10Z  UC0ZV6M2THA81QT9hrVWJG3A   Apex Legends          20   
2  2020-08-11T16:34:06Z  UCYzPXprvl5Y-Sf0g4vX-m6g  jacksepticeye          24   
3  2020-08-11T16:38:55Z  UCbg_UMjlHJg_19SZckaKajg            XXL          10   
4  2020-08-11T15:10:05Z  UCDVPcEbVLQgLZX0Rt6jo34A       Mr. Kate          26   

          trending_date                                               tags  \
0  2020-08-12T00:00:00Z  b

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description'],
      dtype='object')

**US기준이이라서 다 영어권이지만, 한국영상들의 태그나 설명칸에 한국어가 들어가있을 가능성이 높기때문에 정제하는 과정**

In [ ]:
import plotly.express as px

# 필요한 열만 선택
selected_columns = ['video_id', 'title', 'publishedAt', 'channelTitle', 'categoryId', 'trending_date', 'tags', 'view_count', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'description']

df = df[selected_columns]

# title, tags, description 중 하나라도 한글이 포함된 경우 필터링
korean_ratings_videos = df[(df['ratings_disabled'] == True) &
                           ((df['title'].str.contains('[\u3131-\u3163\uac00-\ud7a3]+', regex=True, na=False)) |
                            (df['tags'].str.contains('[\u3131-\u3163\uac00-\ud7a3]+', regex=True, na=False)) |
                            (df['description'].str.contains('[\u3131-\u3163\uac00-\ud7a3]+', regex=True, na=False)))]

# 날짜별로 그룹화하여 각 날짜별 한국 레이팅 영상 수 및 뷰 카운트 계산
korean_ratings_by_date = korean_ratings_videos.groupby('trending_date').agg({'video_id': 'count', 'view_count': 'sum'}).reset_index()
korean_ratings_by_date.columns = ['trending_date', 'count', 'total_view_count']
# 각 날짜별로 한국 레이팅 영상들의 목록 및 뷰 카운트 출력
for index, row in korean_ratings_by_date.iterrows():
    date = row['trending_date']
    count = row['count']
    total_view_count = row['total_view_count']

    videos_on_date = korean_ratings_videos[korean_ratings_videos['trending_date'] == date]

    print(f"\n날짜: {date}")
    print(f"한국 레이팅 영상 수: {count}")
    print(f"총 뷰 카운트: {total_view_count}")
    print("영상 목록:")
    print(videos_on_date[['video_id', 'title', 'view_count']])
    print("=" * 50)


날짜: 2020-08-19T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 958787
영상 목록:
         video_id                         title  view_count
1439  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction      958787

날짜: 2020-08-20T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1122081
영상 목록:
         video_id                         title  view_count
1672  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1122081

날짜: 2020-08-21T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1213461
영상 목록:
         video_id                         title  view_count
1895  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1213461

날짜: 2020-08-22T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1279156
영상 목록:
         video_id                         title  view_count
2115  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1279156

날짜: 2020-08-23T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1329943
영상 목록:
         video_id                         title  view_count
2331  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1329943

날짜: 2020-08-24T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1363214
영상 목록:
         vi

In [ ]:
# 날짜별로 그룹화하여 각 날짜별 한국 레이팅 영상 수 및 뷰 카운트 계산
korean_ratings_by_date = korean_ratings_videos.groupby(['trending_date', 'categoryId']).agg({'video_id': 'count', 'view_count': 'sum'}).reset_index()

# 시각화
fig = px.line(korean_ratings_by_date,
              x='trending_date',
              y='view_count',
              color='categoryId',
              title='날짜별 한국 레이팅 영상의 카테고리별 뷰 카운트',
              labels={'view_count': '조회수', 'categoryId': '카테고리 ID', 'trending_date': '트렌딩 날짜'})

fig.show()

**캐나다 인기 급상승 동영상 데이터 정제하기**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/CA_youtube_trending_data.csv",sep=',',
                 engine='python',
                 header=0,encoding="UTF-8",error_bad_lines=False)
# 데이터프레임 확인
print(df.head())
df.columns

<ipython-input-35-a0b76bc8b720>:3: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.





      video_id                                              title  \
0  KX06ksuS6Xo  Diljit Dosanjh: CLASH (Official) Music Video |...   
1  J78aPJ3VyNs  I left youtube for a month and THIS is what ha...   
2  M9Pmf9AB4Mo  Apex Legends | Stories from the Outlands – “Th...   
3  3C66w5Z0ixs                 I ASKED HER TO BE MY GIRLFRIEND...   
4  VIUo6yapDbc  Ultimate DIY Home Movie Theater for The LaBran...   

            publishedAt                 channelId    channelTitle  categoryId  \
0  2020-08-11T07:30:02Z  UCZRdNleCgW-BGUJf-bbjzQg  Diljit Dosanjh          10   
1  2020-08-11T16:34:06Z  UCYzPXprvl5Y-Sf0g4vX-m6g   jacksepticeye          24   
2  2020-08-11T17:00:10Z  UC0ZV6M2THA81QT9hrVWJG3A    Apex Legends          20   
3  2020-08-11T19:20:14Z  UCvtRTOMP2TqYqu51xNrqAzg        Brawadis          22   
4  2020-08-11T15:10:05Z  UCDVPcEbVLQgLZX0Rt6jo34A        Mr. Kate          26   

          trending_date                                               tags  \
0  2020-08-12T00:00:

Skipping line 9319: unexpected end of data


Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description'],
      dtype='object')

In [ ]:
import plotly.express as px

# 필요한 열만 선택
selected_columns = ['video_id', 'title', 'publishedAt', 'channelTitle', 'categoryId', 'trending_date', 'tags', 'view_count', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'description']

df = df[selected_columns]

# title, tags, description 중 하나라도 한글이 포함된 경우 필터링
korean_ratings_videos = df[(df['ratings_disabled'] == True) &
                           ((df['title'].str.contains('[\u3131-\u3163\uac00-\ud7a3]+', regex=True, na=False)) |
                            (df['tags'].str.contains('[\u3131-\u3163\uac00-\ud7a3]+', regex=True, na=False)) |
                            (df['description'].str.contains('[\u3131-\u3163\uac00-\ud7a3]+', regex=True, na=False)))]

# 날짜별로 그룹화하여 각 날짜별 한국 레이팅 영상 수 및 뷰 카운트 계산
korean_ratings_by_date = korean_ratings_videos.groupby('trending_date').agg({'video_id': 'count', 'view_count': 'sum'}).reset_index()
korean_ratings_by_date.columns = ['trending_date', 'count', 'total_view_count']
# 각 날짜별로 한국 레이팅 영상들의 목록 및 뷰 카운트 출력
for index, row in korean_ratings_by_date.iterrows():
    date = row['trending_date']
    count = row['count']
    total_view_count = row['total_view_count']

    videos_on_date = korean_ratings_videos[korean_ratings_videos['trending_date'] == date]

    print(f"\n날짜: {date}")
    print(f"한국 레이팅 영상 수: {count}")
    print(f"총 뷰 카운트: {total_view_count}")
    print("영상 목록:")
    print(videos_on_date[['video_id', 'title', 'view_count']])
    print("=" * 50)


날짜: 2020-08-19T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 958787
영상 목록:
         video_id                         title  view_count
1439  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction      958787

날짜: 2020-08-20T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1122081
영상 목록:
         video_id                         title  view_count
1686  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1122081

날짜: 2020-08-21T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1213461
영상 목록:
         video_id                         title  view_count
1907  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1213461

날짜: 2020-08-22T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1279156
영상 목록:
         video_id                         title  view_count
2120  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1279156

날짜: 2020-08-23T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1329943
영상 목록:
         video_id                         title  view_count
2345  RHPRUe-aZHA  SuperM 슈퍼엠 '100' MV Reaction     1329943

날짜: 2020-09-05T00:00:00Z
한국 레이팅 영상 수: 1
총 뷰 카운트: 1497505
영상 목록:
         vi

In [ ]:
# 날짜별로 그룹화하여 각 날짜별 한국 레이팅 영상 수 및 뷰 카운트 계산
korean_ratings_by_date = korean_ratings_videos.groupby(['trending_date', 'categoryId']).agg({'video_id': 'count', 'view_count': 'sum'}).reset_index()

# 시각화
fig = px.line(korean_ratings_by_date,
              x='trending_date',
              y='view_count',
              color='categoryId',
              title='날짜별 한국 레이팅 영상의 카테고리별 뷰 카운트',
              labels={'view_count': '조회수', 'categoryId': '카테고리 ID', 'trending_date': '트렌딩 날짜'})

fig.show()

코드에서 “<를 인식하지 못하여서 csv를 수정하였으나 csv가 이상하게 저장되거나 , 계속 오류가 발생하였다. 그래서 오류가나는 곳을 스킵을했더니 계속 런타임을 재시작하면 됐다가 안됐다가, 값이 들어갔다가 안들어갔다 하는 버그가 있었다.

한국 컨텐츠가 미국과 캐나다에서 인기를 얻고 있는 것은 흥미로운 현상이다. 특히 음악 컨텐츠가 두각을 내고 있다는 것은 K-pop 등 한국 음악의 국제적 성공을 반영하는 것으로 보인다.. 이러한 성공은 한국 음악산업과 아티스트들의 글로벌 영향력이 계속해서 성장하고 있다는 증거일 수 있다.

그래프를 통해 이러한 추세를 확인하고 있다면, 한국 컨텐츠 산업의 성장과 국제적 인지도가 높아지고 있다는 것을 나타낼 수 있다. 이는 한국의 엔터테인먼트 기업, 가수, 그리고 음악 프로듀서들이 해외 시장을 탐구하고 성공적인 마케팅 및 홍보를 진행하고 있음을 시사한다.

더 많은 데이터와 분석을 통해 이러한 추세를 더 자세히 이해할 수 있으며, 한국 컨텐츠 산업의 글로벌 영향력이 앞으로도 계속해서 확대될 것으로 예상된다. 이는 한국의 문화와 예술이 국제 무대에서 더 많이 빛날 수 있는 기회를 의미한다.